In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from datetime import datetime
import pandas as pd

from echolalia.parser import WhatsAppParser

In [3]:
chat_log_filename = "data/_chat.txt"
parser = WhatsAppParser()

messages = parser.parse_chat_log(chat_log_filename)

In [4]:
messages

[{'timestamp': datetime.datetime(2022, 1, 17, 23, 15, 56),
  'user': 'Cat',
  'message': '\u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.',
  'exception': None,
  'chatline': '[1/17/22, 11:15:56\u202fPM] Cat: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.'},
 {'timestamp': datetime.datetime(2022, 1, 17, 23, 15, 56),
  'user': 'Selwyn-Lloyd McPherson',
  'message': 'Hello?!',
  'exception': None,
  'chatline': '[1/17/22, 11:15:56\u202fPM] Selwyn-Lloyd McPherson: Hello?!'},
 {'timestamp': datetime.datetime(2022, 1, 17, 23, 16, 26),
  'user': 'Cat',
  'message': 'HI',
  'exception': None,
  'chatline': '[1/17/22, 11:16:26\u202fPM] Cat: HI'},
 {'timestamp': datetime.datetime(2022, 1, 17, 23, 17, 24),
  'user': 'Cat',
  'message': 'oh good i do have it on my computer already',
  'exception': None,
  'chatline': '[1/17/22, 11:17:24\u20

In [6]:
p_messages = pd.DataFrame(messages)

In [7]:
p_messages

,timestamp,user,message,exception,chatline
0,2022-01-17 23:15:56,Cat,‎Messages and calls are end-to-end encrypted. ...,None,"[1/17/22, 11:15:56 PM] Cat: ‎Messages and call..."
1,2022-01-17 23:15:56,Selwyn-Lloyd McPherson,Hello?!,None,"[1/17/22, 11:15:56 PM] Selwyn-Lloyd McPherson:..."
2,2022-01-17 23:16:26,Cat,HI,None,"[1/17/22, 11:16:26 PM] Cat: HI"
3,2022-01-17 23:17:24,Cat,oh good i do have it on my computer already,None,"[1/17/22, 11:17:24 PM] Cat: oh good i do have ..."
4,2022-01-17 23:17:24,Selwyn-Lloyd McPherson,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd...",None,"[1/17/22, 11:17:24 PM] Selwyn-Lloyd McPherson:..."
...,...,...,...,...,...
53904,2024-08-06 12:44:43,Cat,Cough cough wheezeeeee cough,None,"[8/6/24, 12:44:43 PM] Cat: Cough cough wheezee..."
53905,2024-08-08 13:13:14,Cat,My turn in the er,None,"[8/8/24, 1:13:14 PM] Cat: My turn in the er"
53906,2024-08-08 16:41:41,Cat,Ya girls got cirrhosis,None,"[8/8/24, 4:41:41 PM] Cat: Ya girls got cirrhosis"
53907,2024-08-08 16:41:49,Cat,Among other things,None,"[8/8/24, 4:41:49 PM] Cat: Among other things"
